# Importing libraries

In [1]:
import os
import sys
import glob
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
from pyspark.sql import SparkSession,functions, Window
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark.pandas as ps


# Starting Spark Session and Creating a Spark Session Called Pizza

In [3]:
spark = SparkSession.builder \
        .master("local[1]") \
        .appName("PizzaRestaurant") \
        .config("spark.some.config.option", "config-value") \
        .enableHiveSupport() \
        .getOrCreate()

In [4]:
spark

# Browsing .csv PizzaRestaurant files

In [5]:
path = r"D:\My commercial projects\PySpark\Project 4 PySpark\Pizza Place Sales"


if os.path.exists(path):
    print(f'The file {path} exists')
else: 
    print(f"The file {path} does not exist ")

filenames = glob.glob(path + '\*.csv')

for file in filenames:
    print("\nFile names:", file)


The file D:\My commercial projects\PySpark\Project 4 PySpark\Pizza Place Sales exists

File names: D:\My commercial projects\PySpark\Project 4 PySpark\Pizza Place Sales\orders.csv

File names: D:\My commercial projects\PySpark\Project 4 PySpark\Pizza Place Sales\order_details.csv

File names: D:\My commercial projects\PySpark\Project 4 PySpark\Pizza Place Sales\pizzas.csv

File names: D:\My commercial projects\PySpark\Project 4 PySpark\Pizza Place Sales\pizza_types.csv


# Reading csv files into Spark DataFrames

In [6]:
PathOrders = 'D:\My commercial projects\PySpark\Project 4 PySpark\Pizza Place Sales\orders.csv'
PathOrderDetails = 'D:\My commercial projects\PySpark\Project 4 PySpark\Pizza Place Sales\order_details.csv'
PathPizzas = 'D:\My commercial projects\PySpark\Project 4 PySpark\Pizza Place Sales\pizzas.csv'
PathPizzaTypes = 'D:\My commercial projects\PySpark\Project 4 PySpark\Pizza Place Sales\pizza_types.csv'

dfPizzaOrders = spark.read.csv(PathOrders, header=True, inferSchema=True)
dfPizzaOrderDetails = spark.read.csv(PathOrderDetails, header=True, inferSchema=True)
dfPizzas = spark.read.csv(PathPizzas, header=True, inferSchema=True)
dfPizzaTypes = spark.read.csv(PathPizzaTypes, header=True, inferSchema=True)


# Data Exploration

In [7]:
dfPizzaOrders.show(truncate=False)

+--------+----------+-------------------+
|order_id|date      |time               |
+--------+----------+-------------------+
|1       |2015-01-01|2023-06-15 11:38:36|
|2       |2015-01-01|2023-06-15 11:57:40|
|3       |2015-01-01|2023-06-15 12:12:28|
|4       |2015-01-01|2023-06-15 12:16:31|
|5       |2015-01-01|2023-06-15 12:21:30|
|6       |2015-01-01|2023-06-15 12:29:36|
|7       |2015-01-01|2023-06-15 12:50:37|
|8       |2015-01-01|2023-06-15 12:51:37|
|9       |2015-01-01|2023-06-15 12:52:01|
|10      |2015-01-01|2023-06-15 13:00:15|
|11      |2015-01-01|2023-06-15 13:02:59|
|12      |2015-01-01|2023-06-15 13:04:41|
|13      |2015-01-01|2023-06-15 13:11:55|
|14      |2015-01-01|2023-06-15 13:14:19|
|15      |2015-01-01|2023-06-15 13:33:00|
|16      |2015-01-01|2023-06-15 13:34:07|
|17      |2015-01-01|2023-06-15 13:53:00|
|18      |2015-01-01|2023-06-15 13:57:08|
|19      |2015-01-01|2023-06-15 13:59:09|
|20      |2015-01-01|2023-06-15 14:03:08|
+--------+----------+-------------

In [8]:
dfPizzaOrderDetails.show(truncate=False)

+----------------+--------+--------------+--------+
|order_details_id|order_id|pizza_id      |quantity|
+----------------+--------+--------------+--------+
|1               |1       |hawaiian_m    |1       |
|2               |2       |classic_dlx_m |1       |
|3               |2       |five_cheese_l |1       |
|4               |2       |ital_supr_l   |1       |
|5               |2       |mexicana_m    |1       |
|6               |2       |thai_ckn_l    |1       |
|7               |3       |ital_supr_m   |1       |
|8               |3       |prsc_argla_l  |1       |
|9               |4       |ital_supr_m   |1       |
|10              |5       |ital_supr_m   |1       |
|11              |6       |bbq_ckn_s     |1       |
|12              |6       |the_greek_s   |1       |
|13              |7       |spinach_supr_s|1       |
|14              |8       |spinach_supr_s|1       |
|15              |9       |classic_dlx_s |1       |
|16              |9       |green_garden_s|1       |
|17         

In [9]:
dfPizzas.show(truncate=False)

+-------------+-------------+----+-----+
|pizza_id     |pizza_type_id|size|price|
+-------------+-------------+----+-----+
|bbq_ckn_s    |bbq_ckn      |S   |12.75|
|bbq_ckn_m    |bbq_ckn      |M   |16.75|
|bbq_ckn_l    |bbq_ckn      |L   |20.75|
|cali_ckn_s   |cali_ckn     |S   |12.75|
|cali_ckn_m   |cali_ckn     |M   |16.75|
|cali_ckn_l   |cali_ckn     |L   |20.75|
|ckn_alfredo_s|ckn_alfredo  |S   |12.75|
|ckn_alfredo_m|ckn_alfredo  |M   |16.75|
|ckn_alfredo_l|ckn_alfredo  |L   |20.75|
|ckn_pesto_s  |ckn_pesto    |S   |12.75|
|ckn_pesto_m  |ckn_pesto    |M   |16.75|
|ckn_pesto_l  |ckn_pesto    |L   |20.75|
|southw_ckn_s |southw_ckn   |S   |12.75|
|southw_ckn_m |southw_ckn   |M   |16.75|
|southw_ckn_l |southw_ckn   |L   |20.75|
|thai_ckn_s   |thai_ckn     |S   |12.75|
|thai_ckn_m   |thai_ckn     |M   |16.75|
|thai_ckn_l   |thai_ckn     |L   |20.75|
|big_meat_s   |big_meat     |S   |12.0 |
|big_meat_m   |big_meat     |M   |16.0 |
+-------------+-------------+----+-----+
only showing top

In [10]:
dfPizzaTypes.show(truncate=False)

+-------------+------------------------------------------+--------+--------------------------------------------------------------------------------------------+
|pizza_type_id|name                                      |category|ingredients                                                                                 |
+-------------+------------------------------------------+--------+--------------------------------------------------------------------------------------------+
|bbq_ckn      |The Barbecue Chicken Pizza                |Chicken |Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce         |
|cali_ckn     |The California Chicken Pizza              |Chicken |Chicken, Artichoke, Spinach, Garlic, Jalapeno Peppers, Fontina Cheese, Gouda Cheese         |
|ckn_alfredo  |The Chicken Alfredo Pizza                 |Chicken |Chicken, Red Onions, Red Peppers, Mushrooms, Asiago Cheese, Alfredo Sauce                   |
|ckn_pesto    |The Chicken Pesto P

# Combining DataFrames

In [11]:
dfPizzaOrdersAll = dfPizzaOrders.join(dfPizzaOrderDetails,dfPizzaOrders.order_id == dfPizzaOrderDetails.order_id, "inner" )
dfPizzaOrdersAll.show(truncate=False)

+--------+----------+-------------------+----------------+--------+--------------+--------+
|order_id|date      |time               |order_details_id|order_id|pizza_id      |quantity|
+--------+----------+-------------------+----------------+--------+--------------+--------+
|1       |2015-01-01|2023-06-15 11:38:36|1               |1       |hawaiian_m    |1       |
|2       |2015-01-01|2023-06-15 11:57:40|2               |2       |classic_dlx_m |1       |
|2       |2015-01-01|2023-06-15 11:57:40|3               |2       |five_cheese_l |1       |
|2       |2015-01-01|2023-06-15 11:57:40|4               |2       |ital_supr_l   |1       |
|2       |2015-01-01|2023-06-15 11:57:40|5               |2       |mexicana_m    |1       |
|2       |2015-01-01|2023-06-15 11:57:40|6               |2       |thai_ckn_l    |1       |
|3       |2015-01-01|2023-06-15 12:12:28|7               |3       |ital_supr_m   |1       |
|3       |2015-01-01|2023-06-15 12:12:28|8               |3       |prsc_argla_l 

In [12]:
dfPizzaOrdersAll = dfPizzaOrdersAll.select("pizza_id","quantity","date","time")
dfPizzaOrdersAll.show()

+--------------+--------+----------+-------------------+
|      pizza_id|quantity|      date|               time|
+--------------+--------+----------+-------------------+
|    hawaiian_m|       1|2015-01-01|2023-06-15 11:38:36|
| classic_dlx_m|       1|2015-01-01|2023-06-15 11:57:40|
| five_cheese_l|       1|2015-01-01|2023-06-15 11:57:40|
|   ital_supr_l|       1|2015-01-01|2023-06-15 11:57:40|
|    mexicana_m|       1|2015-01-01|2023-06-15 11:57:40|
|    thai_ckn_l|       1|2015-01-01|2023-06-15 11:57:40|
|   ital_supr_m|       1|2015-01-01|2023-06-15 12:12:28|
|  prsc_argla_l|       1|2015-01-01|2023-06-15 12:12:28|
|   ital_supr_m|       1|2015-01-01|2023-06-15 12:16:31|
|   ital_supr_m|       1|2015-01-01|2023-06-15 12:21:30|
|     bbq_ckn_s|       1|2015-01-01|2023-06-15 12:29:36|
|   the_greek_s|       1|2015-01-01|2023-06-15 12:29:36|
|spinach_supr_s|       1|2015-01-01|2023-06-15 12:50:37|
|spinach_supr_s|       1|2015-01-01|2023-06-15 12:51:37|
| classic_dlx_s|       1|2015-0

In [13]:
dfPizzaAll = dfPizzas.join(dfPizzaTypes,dfPizzas.pizza_type_id == dfPizzaTypes.pizza_type_id, "inner" )
dfPizzaAll = dfPizzaAll.select("pizza_id","size","price","name","category","ingredients")
dfPizzaAll.show()


+-------------+----+-----+--------------------+--------+--------------------+
|     pizza_id|size|price|                name|category|         ingredients|
+-------------+----+-----+--------------------+--------+--------------------+
|    bbq_ckn_l|   L|20.75|The Barbecue Chic...| Chicken|Barbecued Chicken...|
|    bbq_ckn_m|   M|16.75|The Barbecue Chic...| Chicken|Barbecued Chicken...|
|    bbq_ckn_s|   S|12.75|The Barbecue Chic...| Chicken|Barbecued Chicken...|
|   cali_ckn_l|   L|20.75|The California Ch...| Chicken|Chicken, Artichok...|
|   cali_ckn_m|   M|16.75|The California Ch...| Chicken|Chicken, Artichok...|
|   cali_ckn_s|   S|12.75|The California Ch...| Chicken|Chicken, Artichok...|
|ckn_alfredo_l|   L|20.75|The Chicken Alfre...| Chicken|Chicken, Red Onio...|
|ckn_alfredo_m|   M|16.75|The Chicken Alfre...| Chicken|Chicken, Red Onio...|
|ckn_alfredo_s|   S|12.75|The Chicken Alfre...| Chicken|Chicken, Red Onio...|
|  ckn_pesto_l|   L|20.75|The Chicken Pesto...| Chicken|Chicken,

In [14]:
dfPizza = dfPizzaAll.join(dfPizzaOrdersAll,dfPizzaAll.pizza_id == dfPizzaOrdersAll.pizza_id, "left" )
dfPizza = dfPizza.select(col("name").alias("Name"), 
                col("category").alias("Category"),
                col("ingredients").alias("Ingredients"),
                col("size").alias("Size"),
                col("price").alias("Price"),
                col("quantity").alias("Quantity"),
                col("date").alias("Date"),
                col("time").alias("Time"))

dfPizza.show(truncate=False)


+--------------------------+--------+-----------------------------------------------------------------------------------+----+-----+--------+----------+-------------------+
|Name                      |Category|Ingredients                                                                        |Size|Price|Quantity|Date      |Time               |
+--------------------------+--------+-----------------------------------------------------------------------------------+----+-----+--------+----------+-------------------+
|The Barbecue Chicken Pizza|Chicken |Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce|L   |20.75|1       |2015-01-01|2023-06-15 13:02:59|
|The Barbecue Chicken Pizza|Chicken |Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce|L   |20.75|1       |2015-01-01|2023-06-15 13:53:00|
|The Barbecue Chicken Pizza|Chicken |Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce|L   |20.75|1  

# Data Cleaning

### Looking for null values

In [15]:
dfPizzaNull = dfPizza.select("Name", "Category", "Ingredients", "Size", "Price", "Quantity")

dfPizzaNull.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in dfPizzaNull.columns]
   ).show()

+----+--------+-----------+----+-----+--------+
|Name|Category|Ingredients|Size|Price|Quantity|
+----+--------+-----------+----+-----+--------+
|   0|       0|          0|   0|    0|       5|
+----+--------+-----------+----+-----+--------+



##### There are a few Null values in Quantity, but it means that there were no orders for these pizza

In [16]:
dfPizza.filter(col("Quantity").isNull()).show()

+--------------------+--------+--------------------+----+-----+--------+----+----+
|                Name|Category|         Ingredients|Size|Price|Quantity|Date|Time|
+--------------------+--------+--------------------+----+-----+--------+----+----+
|The Four Cheese P...|  Veggie|Ricotta Cheese, G...|   S|11.75|    null|null|null|
|  The Big Meat Pizza| Classic|Bacon, Pepperoni,...|   L| 20.5|    null|null|null|
|The Five Cheese P...|  Veggie|Mozzarella Cheese...|   S| 12.5|    null|null|null|
|  The Big Meat Pizza| Classic|Bacon, Pepperoni,...|   M| 16.0|    null|null|null|
|The Five Cheese P...|  Veggie|Mozzarella Cheese...|   M| 15.5|    null|null|null|
+--------------------+--------+--------------------+----+-----+--------+----+----+



### Dealing with duplicate values

In [17]:
print("Counting rows {}".format(dfPizza.count()))

Counting rows 48625


In [18]:
dfPizzaDuplicates = dfPizza.groupBy(dfPizza.columns).count().filter(col("count") > 1)
dfPizzaDuplicates.show()

+----+--------+-----------+----+-----+--------+----+----+-----+
|Name|Category|Ingredients|Size|Price|Quantity|Date|Time|count|
+----+--------+-----------+----+-----+--------+----+----+-----+
+----+--------+-----------+----+-----+--------+----+----+-----+



### Browsing for distinct values in each column

In [19]:
print("Counting distinct rows {}".format(dfPizza.distinct().count()))

Counting distinct rows 48625


In [20]:
dfPizza.agg(*(countDistinct(col(c)).alias(c) for c in dfPizza.columns)).show()

+----+--------+-----------+----+-----+--------+----+-----+
|Name|Category|Ingredients|Size|Price|Quantity|Date| Time|
+----+--------+-----------+----+-----+--------+----+-----+
|  32|       4|         32|   5|   27|       4| 358|16382|
+----+--------+-----------+----+-----+--------+----+-----+



### Columns

In [21]:
print("There are {} columns in the DataFrame and their names are {}".format(len(dfPizza.columns),
                                                                              dfPizza.columns))

There are 8 columns in the DataFrame and their names are ['Name', 'Category', 'Ingredients', 'Size', 'Price', 'Quantity', 'Date', 'Time']


### Datatypes

In [22]:
dfPizza.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Ingredients: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: timestamp (nullable = true)



In [23]:
dfPizza = dfPizza.withColumn("Price",col("Price").cast(DecimalType(18,2)))

### Summary

In [24]:
dfPizza.describe("Price").show()

+-------+-----------------+
|summary|            Price|
+-------+-----------------+
|  count|            48625|
|   mean|        16.494004|
| stddev|3.621761197368334|
|    min|             9.75|
|    max|            35.95|
+-------+-----------------+



In [25]:
dfPizza.dtypes

[('Name', 'string'),
 ('Category', 'string'),
 ('Ingredients', 'string'),
 ('Size', 'string'),
 ('Price', 'decimal(18,2)'),
 ('Quantity', 'int'),
 ('Date', 'date'),
 ('Time', 'timestamp')]

In [26]:
print("Shape of DataFrame: {}, {} ".format(dfPizza.count(),
                                          len(dfPizza.columns)))

Shape of DataFrame: 48625, 8 


# Data Analysis

### Creating Total Amount

In [27]:
def CreateMultpiplication(x, y):
    return x * y

dfPizza = dfPizza.withColumn("Total Amount",CreateMultpiplication(col("Quantity"),col("Price")) )
dfPizza = dfPizza.select('Name',
 'Category',
 'Ingredients',
 'Size',
 'Price',
 'Quantity',
 'Total Amount',
 'Date',
 'Time')

dfPizza.show(truncate=False)

+-------------------------+--------+-----------------------------------------------------------------------+----+-----+--------+------------+----------+-------------------+
|Name                     |Category|Ingredients                                                            |Size|Price|Quantity|Total Amount|Date      |Time               |
+-------------------------+--------+-----------------------------------------------------------------------+----+-----+--------+------------+----------+-------------------+
|The Italian Supreme Pizza|Supreme |Calabrese Salami, Capocollo, Tomatoes, Red Onions, Green Olives, Garlic|S   |12.50|1       |12.50       |2015-12-30|2023-06-15 12:45:20|
|The Italian Supreme Pizza|Supreme |Calabrese Salami, Capocollo, Tomatoes, Red Onions, Green Olives, Garlic|S   |12.50|1       |12.50       |2015-12-27|2023-06-15 16:03:36|
|The Italian Supreme Pizza|Supreme |Calabrese Salami, Capocollo, Tomatoes, Red Onions, Green Olives, Garlic|S   |12.50|1       |12.50  

### Creating upper case of Category

In [28]:
def upperCase(str):
    return str.upper()

upperCaseUDF = udf(lambda z:upperCase(z), StringType())

dfPizza = dfPizza.withColumn("Category",upperCaseUDF(col("Category")))
dfPizza.show(truncate=False)

+--------------------------+--------+-----------------------------------------------------------------------------------+----+-----+--------+------------+----------+-------------------+
|Name                      |Category|Ingredients                                                                        |Size|Price|Quantity|Total Amount|Date      |Time               |
+--------------------------+--------+-----------------------------------------------------------------------------------+----+-----+--------+------------+----------+-------------------+
|The Barbecue Chicken Pizza|CHICKEN |Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce|L   |20.75|2       |41.50       |2015-12-31|2023-06-15 19:05:04|
|The Barbecue Chicken Pizza|CHICKEN |Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce|L   |20.75|1       |20.75       |2015-12-31|2023-06-15 18:52:10|
|The Barbecue Chicken Pizza|CHICKEN |Barbecued Chicken, Red Peppers, G

### Summing of Total Amount by Category and Pizza Name

In [29]:
dfPizzaCategoryByTotalAmount = dfPizza.groupBy("Category").sum("Total Amount").orderBy(desc("sum(Total Amount)"))
dfPizzaCategoryByTotalAmount.select("Category",col("sum(Total Amount)").alias("Total Amount")).show(truncate=False)

+--------+------------+
|Category|Total Amount|
+--------+------------+
|CLASSIC |220053.10   |
|SUPREME |208197.00   |
|CHICKEN |195919.50   |
|VEGGIE  |193690.45   |
+--------+------------+



In [30]:
dfPizzaPizzaNameByTotalAmount = dfPizza.groupBy("Name").sum("Total Amount")
dfPizzaPizzaNameByTotalAmount = dfPizzaPizzaNameByTotalAmount.select("Name",col("sum(Total Amount)").alias("Total Amount"))
windowSpec = Window.orderBy(desc("Total Amount"))
dfPizzaPizzaNameByTotalAmount.withColumn("Ranking", row_number().over(windowSpec)).show(truncate=False)

+---------------------------------+------------+-------+
|Name                             |Total Amount|Ranking|
+---------------------------------+------------+-------+
|The Thai Chicken Pizza           |43434.25    |1      |
|The Barbecue Chicken Pizza       |42768.00    |2      |
|The California Chicken Pizza     |41409.50    |3      |
|The Classic Deluxe Pizza         |38180.50    |4      |
|The Spicy Italian Pizza          |34831.25    |5      |
|The Southwest Chicken Pizza      |34705.75    |6      |
|The Italian Supreme Pizza        |33476.75    |7      |
|The Hawaiian Pizza               |32273.25    |8      |
|The Four Cheese Pizza            |32265.70    |9      |
|The Sicilian Pizza               |30940.50    |10     |
|The Pepperoni Pizza              |30161.75    |11     |
|The Greek Pizza                  |28454.10    |12     |
|The Mexicana Pizza               |26780.75    |13     |
|The Five Cheese Pizza            |26066.50    |14     |
|The Pepper Salami Pizza       

### Grouping of Pizza Name 

In [31]:
dfPizzaPizzaNameByTotalAmount.withColumn("Grouping",ntile(5).over(windowSpec)).show(truncate=False)

dfPizzaPizzaNameByTotalAmountGrouping =  dfPizzaPizzaNameByTotalAmount.withColumn("Grouping",ntile(5).over(windowSpec))


+---------------------------------+------------+--------+
|Name                             |Total Amount|Grouping|
+---------------------------------+------------+--------+
|The Thai Chicken Pizza           |43434.25    |1       |
|The Barbecue Chicken Pizza       |42768.00    |1       |
|The California Chicken Pizza     |41409.50    |1       |
|The Classic Deluxe Pizza         |38180.50    |1       |
|The Spicy Italian Pizza          |34831.25    |1       |
|The Southwest Chicken Pizza      |34705.75    |1       |
|The Italian Supreme Pizza        |33476.75    |1       |
|The Hawaiian Pizza               |32273.25    |2       |
|The Four Cheese Pizza            |32265.70    |2       |
|The Sicilian Pizza               |30940.50    |2       |
|The Pepperoni Pizza              |30161.75    |2       |
|The Greek Pizza                  |28454.10    |2       |
|The Mexicana Pizza               |26780.75    |2       |
|The Five Cheese Pizza            |26066.50    |2       |
|The Pepper Sa

In [32]:
dfPizzaPizzaNameByTotalAmountGrouping.filter(dfPizzaPizzaNameByTotalAmountGrouping["Grouping"] == 1).show(truncate=False)

+----------------------------+------------+--------+
|Name                        |Total Amount|Grouping|
+----------------------------+------------+--------+
|The Thai Chicken Pizza      |43434.25    |1       |
|The Barbecue Chicken Pizza  |42768.00    |1       |
|The California Chicken Pizza|41409.50    |1       |
|The Classic Deluxe Pizza    |38180.50    |1       |
|The Spicy Italian Pizza     |34831.25    |1       |
|The Southwest Chicken Pizza |34705.75    |1       |
|The Italian Supreme Pizza   |33476.75    |1       |
+----------------------------+------------+--------+



In [33]:
dfPizzaPizzaNameByTotalAmountGrouping.filter(dfPizzaPizzaNameByTotalAmountGrouping["Grouping"] == 5).show(truncate=False)

+-------------------------+------------+--------+
|Name                     |Total Amount|Grouping|
+-------------------------+------------+--------+
|The Calabrese Pizza      |15934.25    |5       |
|The Spinach Pesto Pizza  |15596.00    |5       |
|The Mediterranean Pizza  |15360.50    |5       |
|The Spinach Supreme Pizza|15277.75    |5       |
|The Green Garden Pizza   |13955.75    |5       |
|The Brie Carre Pizza     |11588.50    |5       |
+-------------------------+------------+--------+



### Running Total by Month

In [34]:
dfPizzaGrouping = dfPizza.select(year(col("Date")),
               month(col("Date")),
               col("Total Amount"))


dfPizzaGrouping = dfPizzaGrouping.groupBy("year(Date)","month(Date)").sum("Total Amount")
dfPizzaGrouping = dfPizzaGrouping.na.drop()
dfPizzaGrouping = dfPizzaGrouping.select(col("year(Date)").alias("Year"),
                       col("month(Date)").alias("Month"),
                       col("sum(Total Amount)").alias("Total Amount"))

dfPizzaGrouping.orderBy("Month").show(truncate=False)


+----+-----+------------+
|Year|Month|Total Amount|
+----+-----+------------+
|2015|1    |69793.30    |
|2015|2    |65159.60    |
|2015|3    |70397.10    |
|2015|4    |68736.80    |
|2015|5    |71402.75    |
|2015|6    |68230.20    |
|2015|7    |72557.90    |
|2015|8    |68278.25    |
|2015|9    |64180.05    |
|2015|10   |64027.60    |
|2015|11   |70395.35    |
|2015|12   |64701.15    |
+----+-----+------------+



In [35]:
windowSpec = Window.partitionBy().orderBy("Month")
windowval = (Window.partitionBy().orderBy("Month").rangeBetween(Window.unboundedPreceding,0))

dfPizzaGrouping = dfPizzaGrouping.select("Year",
                                         "Month",
                                         concat_ws("-",col("Month"),col("Year")).alias("Month-Year"),
                                         "Total Amount")
dfPizzaGrouping = dfPizzaGrouping.withColumn("Lag", lag(col("Total Amount"), offset=1, default=0).over(windowSpec))
dfPizzaGrouping = dfPizzaGrouping.withColumn("Diff",
                                             when(isnull(col("Total Amount") - col("Lag")),0)
                                                  .otherwise(col("Total Amount") - col("Lag")))
dfPizzaGrouping = dfPizzaGrouping.withColumn("Lead", lead(col("Total Amount"), offset=1,default = 0).over(windowSpec))
dfPizzaGrouping = dfPizzaGrouping.withColumn("Cummluative Sum", sum(col("Total Amount")).over(windowval))

dfPizzaGrouping.select("Month-Year",
                       "Total Amount",
                       "Lag",
                       "Diff",
                       "Lead",
                       "Cummluative Sum").show(truncate=False )




+----------+------------+--------+--------+--------+---------------+
|Month-Year|Total Amount|Lag     |Diff    |Lead    |Cummluative Sum|
+----------+------------+--------+--------+--------+---------------+
|1-2015    |69793.30    |0.00    |69793.30|65159.60|69793.30       |
|2-2015    |65159.60    |69793.30|-4633.70|70397.10|134952.90      |
|3-2015    |70397.10    |65159.60|5237.50 |68736.80|205350.00      |
|4-2015    |68736.80    |70397.10|-1660.30|71402.75|274086.80      |
|5-2015    |71402.75    |68736.80|2665.95 |68230.20|345489.55      |
|6-2015    |68230.20    |71402.75|-3172.55|72557.90|413719.75      |
|7-2015    |72557.90    |68230.20|4327.70 |68278.25|486277.65      |
|8-2015    |68278.25    |72557.90|-4279.65|64180.05|554555.90      |
|9-2015    |64180.05    |68278.25|-4098.20|64027.60|618735.95      |
|10-2015   |64027.60    |64180.05|-152.45 |70395.35|682763.55      |
|11-2015   |70395.35    |64027.60|6367.75 |64701.15|753158.90      |
|12-2015   |64701.15    |70395.35|

### The quantity of orders in terms of Pizza and its size

In [36]:
dfOrderedPizza = dfPizza.na.drop()

dfOrderedPizza = dfOrderedPizza.groupBy("Name").count()

dfOrderedPizza = dfOrderedPizza.select("Name",col("Count").alias("Quantity of Orders")).orderBy(desc("count"))

dfOrderedPizza.show(dfOrderedPizza.count(), False)


+------------------------------------------+------------------+
|Name                                      |Quantity of Orders|
+------------------------------------------+------------------+
|The Classic Deluxe Pizza                  |2416              |
|The Barbecue Chicken Pizza                |2372              |
|The Hawaiian Pizza                        |2370              |
|The Pepperoni Pizza                       |2369              |
|The Thai Chicken Pizza                    |2315              |
|The California Chicken Pizza              |2302              |
|The Spicy Italian Pizza                   |1887              |
|The Sicilian Pizza                        |1887              |
|The Southwest Chicken Pizza               |1885              |
|The Four Cheese Pizza                     |1850              |
|The Italian Supreme Pizza                 |1849              |
|The Big Meat Pizza                        |1811              |
|The Vegetables + Vegetables Pizza      

In [37]:
dfPizzaSize = dfPizza.groupBy("Size").count()
dfPizzaSize.na.drop()
dfPizzaSize.select("Size", col("count").alias("Quantity of Orders")).orderBy(desc("Quantity of Orders")).show(truncate=False)




+----+------------------+
|Size|Quantity of Orders|
+----+------------------+
|L   |18527             |
|M   |15387             |
|S   |14139             |
|XL  |544               |
|XXL |28                |
+----+------------------+



### Quantity of orders in a day and which pizza was ordered most of the time

In [38]:
dfPizzaWithoutTime  = dfPizza.select('Name',
 'Category',
 'Ingredients',
 'Size',
 'Price',
 'Quantity',
 'Total Amount',
 'Date')

dfPizzaDayMonthYear = dfPizzaWithoutTime.withColumn("Day", dayofmonth(col("Date"))) \
        .withColumn("Month", month(col("Date"))) \
        .withColumn("Month Name",date_format("date", "MMMM")) \
        .withColumn("Year", year(col("Date")))

dfPizzaDayMonthYear = dfPizzaDayMonthYear.na.drop()

dfPizzaDayMonthYearQuantity = dfPizzaDayMonthYear.groupBy("Day","Month","Month Name").count().orderBy("Month","Day")

dfPizzaDayMonthYearQuantity = dfPizzaDayMonthYearQuantity.select("Day","Month Name", col("count").alias("Quantity of Orders"))

dfPizzaDayMonthYearQuantity.show(dfPizzaDayMonthYearQuantity.count(), False)

+---+----------+------------------+
|Day|Month Name|Quantity of Orders|
+---+----------+------------------+
|1  |January   |161               |
|2  |January   |160               |
|3  |January   |154               |
|4  |January   |106               |
|5  |January   |121               |
|6  |January   |144               |
|7  |January   |133               |
|8  |January   |171               |
|9  |January   |123               |
|10 |January   |145               |
|11 |January   |114               |
|12 |January   |118               |
|13 |January   |117               |
|14 |January   |144               |
|15 |January   |123               |
|16 |January   |155               |
|17 |January   |122               |
|18 |January   |119               |
|19 |January   |139               |
|20 |January   |139               |
|21 |January   |127               |
|22 |January   |155               |
|23 |January   |149               |
|24 |January   |142               |
|25 |January   |101         

In [39]:
dfPizzaDayNameQuantity = dfPizzaDayMonthYear.groupBy("Day","Month","Month Name","Name").count().orderBy("Month","Day")


dfPizzaDayNameQuantity = dfPizzaDayNameQuantity.select("Day","Month Name","Name",col("count").alias("Quantity of Orders"))

dfPizzaDayNameQuantity.show(30) 

+---+----------+--------------------+------------------+
|Day|Month Name|                Name|Quantity of Orders|
+---+----------+--------------------+------------------+
|  1|   January|The Italian Veget...|                 2|
|  1|   January|The Barbecue Chic...|                11|
|  1|   January|The Pepper Salami...|                 4|
|  1|   January| The Pepperoni Pizza|                 6|
|  1|   January|The Vegetables + ...|                 3|
|  1|   January|The Pepperoni, Mu...|                 6|
|  1|   January|  The Sicilian Pizza|                 4|
|  1|   January|The Mediterranean...|                 2|
|  1|   January| The Calabrese Pizza|                 1|
|  1|   January|The Soppressata P...|                 1|
|  1|   January|The Chicken Pesto...|                 4|
|  1|   January|  The Hawaiian Pizza|                 4|
|  1|   January|The Napolitana Pizza|                 2|
|  1|   January|The Southwest Chi...|                 7|
|  1|   January|The Classic Del

### Total Amount of Pizza in each day

In [40]:
dfPizzaTotalAmountEachDay = dfPizzaDayMonthYear.groupBy("Day","Month","Month Name").sum("Total Amount").orderBy("Month","Day")

dfPizzaTotalAmountEachDay = dfPizzaTotalAmountEachDay.select("Day","Month Name",col("sum(Total Amount)").alias("Total Amount"))
dfPizzaTotalAmountEachDay.show(dfPizzaTotalAmountEachDay.count(), False)

+---+----------+------------+
|Day|Month Name|Total Amount|
+---+----------+------------+
|1  |January   |2713.85     |
|2  |January   |2731.90     |
|3  |January   |2662.40     |
|4  |January   |1755.45     |
|5  |January   |2065.95     |
|6  |January   |2428.95     |
|7  |January   |2202.20     |
|8  |January   |2838.35     |
|9  |January   |2127.35     |
|10 |January   |2463.95     |
|11 |January   |1872.30     |
|12 |January   |1919.05     |
|13 |January   |2049.60     |
|14 |January   |2527.40     |
|15 |January   |1984.80     |
|16 |January   |2594.15     |
|17 |January   |2064.10     |
|18 |January   |1976.85     |
|19 |January   |2387.15     |
|20 |January   |2397.90     |
|21 |January   |2040.55     |
|22 |January   |2496.70     |
|23 |January   |2423.70     |
|24 |January   |2289.25     |
|25 |January   |1617.55     |
|26 |January   |1884.40     |
|27 |January   |2528.05     |
|28 |January   |2016.00     |
|29 |January   |2045.30     |
|30 |January   |2270.30     |
|31 |Janua

In [41]:
dfPizzaDayNameTotalAmount = dfPizzaDayMonthYear.groupBy("Day","Month","Month Name","Name").sum("Total Amount").orderBy("Month","Day")

dfPizzaDayNameTotalAmount = dfPizzaDayNameTotalAmount.select("Day","Month Name","Name",col("sum(Total Amount)").alias("Total Amount"))

dfPizzaDayNameTotalAmount.show(truncate=False)

+---+----------+------------------------------------------+------------+
|Day|Month Name|Name                                      |Total Amount|
+---+----------+------------------------------------------+------------+
|1  |January   |The Chicken Pesto Pizza                   |79.00       |
|1  |January   |The Italian Supreme Pizza                 |269.00      |
|1  |January   |The Hawaiian Pizza                        |50.75       |
|1  |January   |The Barbecue Chicken Pizza                |204.25      |
|1  |January   |The Napolitana Pizza                      |32.50       |
|1  |January   |The Pepperoni Pizza                       |77.75       |
|1  |January   |The Southwest Chicken Pizza               |125.25      |
|1  |January   |The Pepperoni, Mushroom, and Peppers Pizza|82.50       |
|1  |January   |The Classic Deluxe Pizza                  |156.50      |
|1  |January   |The Mediterranean Pizza                   |52.25       |
|1  |January   |The Thai Chicken Pizza             

### Looking for the greatest value of these and minimum and which day

In [42]:
MaxValueTotalAmount = dfPizzaDayNameTotalAmount.agg({"Total Amount": "max"}).collect()[0][0]
MinValueTotalAmount = dfPizzaDayNameTotalAmount.agg({"Total Amount": "min"}).collect()[0][0]

MaxTotalAmount = dfPizzaDayNameTotalAmount.filter(dfPizzaDayNameTotalAmount["Total Amount"] == MaxValueTotalAmount)
MinTotalAmount = dfPizzaDayNameTotalAmount.filter(dfPizzaDayNameTotalAmount["Total Amount"] == MinValueTotalAmount)

unionPizzaDayNameTotalAmount = MaxTotalAmount.union(MinTotalAmount)

unionPizzaDayNameTotalAmount.show(truncate=False)


+---+----------+------------------------+------------+
|Day|Month Name|Name                    |Total Amount|
+---+----------+------------------------+------------+
|27 |November  |The Classic Deluxe Pizza|322.00      |
|17 |June      |The Pepperoni Pizza     |9.75        |
|10 |December  |The Pepperoni Pizza     |9.75        |
+---+----------+------------------------+------------+



In [43]:
MaxValueQty = dfPizzaDayNameQuantity.agg({"Quantity of Orders": "max"}).collect()[0][0]
MinValueQty = dfPizzaDayNameQuantity.agg({"Quantity of Orders": "min"}).collect()[0][0]

MaxQty = dfPizzaDayNameQuantity.filter(dfPizzaDayNameQuantity["Quantity of Orders"] == MaxValueQty)
MinQty = dfPizzaDayNameQuantity.filter(dfPizzaDayNameQuantity["Quantity of Orders"] == MinValueQty)

unionPizzaDayNameQty = MaxQty.union(MinQty)

unionPizzaDayNameQty.show(truncate=False)



+---+----------+------------------------------------------+------------------+
|Day|Month Name|Name                                      |Quantity of Orders|
+---+----------+------------------------------------------+------------------+
|26 |November  |The Hawaiian Pizza                        |21                |
|1  |January   |The Calabrese Pizza                       |1                 |
|1  |January   |The Soppressata Pizza                     |1                 |
|2  |January   |The Green Garden Pizza                    |1                 |
|2  |January   |The Mediterranean Pizza                   |1                 |
|2  |January   |The Chicken Alfredo Pizza                 |1                 |
|2  |January   |The Pepperoni, Mushroom, and Peppers Pizza|1                 |
|3  |January   |The Brie Carre Pizza                      |1                 |
|3  |January   |The Soppressata Pizza                     |1                 |
|3  |January   |The Green Garden Pizza              

In [44]:
MaxQtyOrder = dfOrderedPizza.agg({"Quantity of Orders": "max"}).collect()[0][0]
MinQtyOrder = dfOrderedPizza.agg({"Quantity of Orders": "min"}).collect()[0][0]

MaxQty = dfOrderedPizza.filter(dfOrderedPizza["Quantity of Orders"] == MaxQtyOrder)
MinQty = dfOrderedPizza.filter(dfOrderedPizza["Quantity of Orders"] == MinQtyOrder)

unionPizzaQty = MaxQty.union(MinQty)

unionPizzaQty.show(truncate=False)


+------------------------+------------------+
|Name                    |Quantity of Orders|
+------------------------+------------------+
|The Classic Deluxe Pizza|2416              |
|The Brie Carre Pizza    |480               |
+------------------------+------------------+



In [45]:
MaxTotalAmount = dfPizzaPizzaNameByTotalAmount.agg({"Total Amount": "max"}).collect()[0][0]
MinTotalAmount = dfPizzaPizzaNameByTotalAmount.agg({"Total Amount": "min"}).collect()[0][0]

MaxTA = dfPizzaPizzaNameByTotalAmount.filter(dfPizzaPizzaNameByTotalAmount["Total Amount"] == MaxTotalAmount)
MinTA = dfPizzaPizzaNameByTotalAmount.filter(dfPizzaPizzaNameByTotalAmount["Total Amount"] == MinTotalAmount)

unionPizzaTotalAmount = MaxTA.union(MinTA)

unionPizzaTotalAmount.show(truncate=False)




+----------------------+------------+
|Name                  |Total Amount|
+----------------------+------------+
|The Thai Chicken Pizza|43434.25    |
|The Brie Carre Pizza  |11588.50    |
+----------------------+------------+



### Counting orders in each month in the name of month

In [46]:
dfPizzaMonthName = dfPizza.withColumn("Month Name",date_format("date", "MMMM")).withColumn("Month", month(col("Date")))
dfPizzaMonthName = dfPizzaMonthName.na.drop()

dfPizzaMonthName = dfPizzaMonthName.groupBy("Month Name","Month").count().orderBy("Month")

dfPizzaMonthName.select(col("Month Name").alias("Month"),col("count").alias("Quantity of Orders")).show(truncate=False)

+---------+------------------+
|Month    |Quantity of Orders|
+---------+------------------+
|January  |4156              |
|February |3892              |
|March    |4186              |
|April    |4067              |
|May      |4239              |
|June     |4025              |
|July     |4301              |
|August   |4094              |
|September|3819              |
|October  |3797              |
|November |4185              |
|December |3859              |
+---------+------------------+



### Dealing with pizza without orders

In [47]:
dfPizzaWithoutOrders = dfPizza.filter(dfPizza.Date.isNull())
dfPizzaWithoutOrders.show(truncate=False)
NamePizzaWithoutOrders = dfPizzaWithoutOrders.agg(collect_set("Name")).collect()[0][0]
CategoryPizzaWithoutOrders = dfPizzaWithoutOrders.agg(collect_set("Category")).collect()[0][0]
QuantityOfPizzaWithoutOrders = dfPizzaWithoutOrders.count()


+---------------------+--------+-------------------------------------------------------------------------------------------------+----+-----+--------+------------+----+----+
|Name                 |Category|Ingredients                                                                                      |Size|Price|Quantity|Total Amount|Date|Time|
+---------------------+--------+-------------------------------------------------------------------------------------------------+----+-----+--------+------------+----+----+
|The Four Cheese Pizza|VEGGIE  |Ricotta Cheese, Gorgonzola Piccante Cheese, Mozzarella Cheese, Parmigiano Reggiano Cheese, Garlic|S   |11.75|null    |null        |null|null|
|The Big Meat Pizza   |CLASSIC |Bacon, Pepperoni, Italian Sausage, Chorizo Sausage                                               |L   |20.50|null    |null        |null|null|
|The Five Cheese Pizza|VEGGIE  |Mozzarella Cheese, Provolone Cheese, Smoked Gouda Cheese, Romano Cheese, Blue Cheese, Garlic     |

In [48]:
print(f'The quantity of pizza without orders: {QuantityOfPizzaWithoutOrders}')
print(f'The name of pizza without orders: {NamePizzaWithoutOrders}')
print(f'The category of pizza without orders: {CategoryPizzaWithoutOrders}')

The quantity of pizza without orders: 5
The name of pizza without orders: ['The Big Meat Pizza', 'The Four Cheese Pizza', 'The Five Cheese Pizza']
The category of pizza without orders: ['VEGGIE', 'CLASSIC']


### Calculating the quantity of orders and total amount regarding pizza in each month

In [49]:
dfPizzaDayNameMonthYear = dfPizzaDayMonthYear.withColumn("Month Name",date_format("date", "MMMM"))
dfPizzaMonthQty = dfPizzaDayNameMonthYear.groupBy("Name","Month Name","Month").count()
dfPizzaMonthTotalAmount = dfPizzaDayNameMonthYear.groupBy("Name","Month Name","Month").sum("Total Amount")



### Orders

In [50]:
dfPizzaMonthNameQty = dfPizzaMonthQty.select("Month Name",
                        "Name",
                        col("count").alias("Quantity of Orders")).orderBy("Month")
dfPizzaMonthNameQty.show(truncate=False)

+----------+---------------------------------+------------------+
|Month Name|Name                             |Quantity of Orders|
+----------+---------------------------------+------------------+
|January   |The Spicy Italian Pizza          |147               |
|January   |The Italian Supreme Pizza        |166               |
|January   |The Vegetables + Vegetables Pizza|143               |
|January   |The Sicilian Pizza               |186               |
|January   |The Classic Deluxe Pizza         |189               |
|January   |The Spinach and Feta Pizza       |123               |
|January   |The California Chicken Pizza     |197               |
|January   |The Thai Chicken Pizza           |197               |
|January   |The Mediterranean Pizza          |66                |
|January   |The Four Cheese Pizza            |153               |
|January   |The Spinach Supreme Pizza        |87                |
|January   |The Pepperoni Pizza              |235               |
|January  

### Ranking

In [51]:
windowSpec = Window.partitionBy("Month Name").orderBy(desc("count"))

dfPizzaMonthQtyRanking= dfPizzaMonthQty.withColumn("Ranking"
                           ,row_number().over(windowSpec)).orderBy("Month",
                                                                   desc("count"))

dfPizzaMonthQtyRanking.select("Month Name",
                              "Name",
                              col("count").alias("Quantity of Orders"),
                              "Ranking").show(truncate=False)

+----------+---------------------------------+------------------+-------+
|Month Name|Name                             |Quantity of Orders|Ranking|
+----------+---------------------------------+------------------+-------+
|January   |The Pepperoni Pizza              |235               |1      |
|January   |The Barbecue Chicken Pizza       |206               |2      |
|January   |The California Chicken Pizza     |197               |3      |
|January   |The Thai Chicken Pizza           |197               |4      |
|January   |The Classic Deluxe Pizza         |189               |5      |
|January   |The Sicilian Pizza               |186               |6      |
|January   |The Hawaiian Pizza               |182               |7      |
|January   |The Italian Supreme Pizza        |166               |8      |
|January   |The Four Cheese Pizza            |153               |9      |
|January   |The Spicy Italian Pizza          |147               |10     |
|January   |The Big Meat Pizza        

In [52]:
dfPizzaMonthQtyFirstRanking = dfPizzaMonthQtyRanking.select("Month Name",
                              "Name",
                              col("count").alias("Quantity of Orders"),
                              "Ranking")

dfPizzaMonthQtyFirstRanking.filter(dfPizzaMonthQtyFirstRanking.Ranking == 1).show(truncate=False)

+----------+----------------------------+------------------+-------+
|Month Name|Name                        |Quantity of Orders|Ranking|
+----------+----------------------------+------------------+-------+
|January   |The Pepperoni Pizza         |235               |1      |
|February  |The Pepperoni Pizza         |201               |1      |
|March     |The Barbecue Chicken Pizza  |229               |1      |
|April     |The Classic Deluxe Pizza    |214               |1      |
|May       |The Pepperoni Pizza         |238               |1      |
|June      |The California Chicken Pizza|216               |1      |
|July      |The Pepperoni Pizza         |223               |1      |
|August    |The California Chicken Pizza|225               |1      |
|September |The Classic Deluxe Pizza    |201               |1      |
|October   |The Classic Deluxe Pizza    |193               |1      |
|November  |The Hawaiian Pizza          |219               |1      |
|December  |The Hawaiian Pizza    

### Maximum Quantity of Orders in month:

In [53]:
MaxQtyOrder = dfPizzaMonthNameQty.agg({"Quantity of Orders": "max"}).collect()[0][0]

dfPizzaMonthNameQty.filter(dfPizzaMonthNameQty["Quantity of Orders"] == MaxQtyOrder).show(truncate=False)


+----------+-------------------+------------------+
|Month Name|Name               |Quantity of Orders|
+----------+-------------------+------------------+
|May       |The Pepperoni Pizza|238               |
+----------+-------------------+------------------+



### Total Amount

In [54]:
dfPizzaMonthNameTotalAmount = dfPizzaMonthTotalAmount.select("Month Name",
                        "Name",
                        col("sum(Total Amount)").alias("Total Amount")).orderBy("Month")

dfPizzaMonthNameTotalAmount.show(truncate=False)


+----------+---------------------------------+------------+
|Month Name|Name                             |Total Amount|
+----------+---------------------------------+------------+
|January   |The Spicy Italian Pizza          |2762.00     |
|January   |The Italian Supreme Pizza        |2954.75     |
|January   |The Vegetables + Vegetables Pizza|2289.50     |
|January   |The Sicilian Pizza               |2987.75     |
|January   |The Classic Deluxe Pizza         |2941.50     |
|January   |The Spinach and Feta Pizza       |2047.25     |
|January   |The California Chicken Pizza     |3471.50     |
|January   |The Thai Chicken Pizza           |3657.25     |
|January   |The Mediterranean Pizza          |1147.50     |
|January   |The Four Cheese Pizza            |2671.65     |
|January   |The Spinach Supreme Pizza        |1357.00     |
|January   |The Pepperoni Pizza              |3001.25     |
|January   |The Southwest Chicken Pizza      |2495.50     |
|January   |The Five Cheese Pizza       

### Ranking

In [55]:
dfPizzaMonthTotalAmount.show(truncate=False)

+------------------------------------------+----------+-----+-----------------+
|Name                                      |Month Name|Month|sum(Total Amount)|
+------------------------------------------+----------+-----+-----------------+
|The Spinach Supreme Pizza                 |September |9    |1116.25          |
|The Italian Capocollo Pizza               |February  |2    |2079.00          |
|The Sicilian Pizza                        |July      |7    |2473.75          |
|The Chicken Pesto Pizza                   |November  |11   |1401.50          |
|The Italian Vegetables Pizza              |August    |8    |1295.50          |
|The Sicilian Pizza                        |April     |4    |2434.75          |
|The Four Cheese Pizza                     |January   |1    |2671.65          |
|The Spinach Supreme Pizza                 |March     |3    |1153.25          |
|The Southwest Chicken Pizza               |June      |6    |2584.00          |
|The Hawaiian Pizza                     

In [56]:
windowSpec = Window.partitionBy("Month Name").orderBy(desc("sum(Total Amount)"))

dfPizzaMonthTotalAmountRanking= dfPizzaMonthTotalAmount.withColumn("Ranking"
                           ,row_number().over(windowSpec)).orderBy("Month",
                                                                   desc("sum(Total Amount)"))

dfPizzaMonthTotalAmountRanking.select("Month Name",
                                      "Name",
                                      col("sum(Total Amount)").alias("Total Amount"),
                                      "Ranking").show(truncate=False)

+----------+---------------------------------+------------+-------+
|Month Name|Name                             |Total Amount|Ranking|
+----------+---------------------------------+------------+-------+
|January   |The Barbecue Chicken Pizza       |3770.25     |1      |
|January   |The Thai Chicken Pizza           |3657.25     |2      |
|January   |The California Chicken Pizza     |3471.50     |3      |
|January   |The Pepperoni Pizza              |3001.25     |4      |
|January   |The Sicilian Pizza               |2987.75     |5      |
|January   |The Italian Supreme Pizza        |2954.75     |6      |
|January   |The Classic Deluxe Pizza         |2941.50     |7      |
|January   |The Spicy Italian Pizza          |2762.00     |8      |
|January   |The Four Cheese Pizza            |2671.65     |9      |
|January   |The Five Cheese Pizza            |2553.00     |10     |
|January   |The Greek Pizza                  |2539.40     |11     |
|January   |The Southwest Chicken Pizza      |24

In [57]:
dfPizzaMonthTotalAmountFirstRanking = dfPizzaMonthTotalAmountRanking.select("Month Name",
                                      "Name",
                                      col("sum(Total Amount)").alias("Total Amount"),
                                      "Ranking")

dfPizzaMonthTotalAmountFirstRanking.filter(dfPizzaMonthTotalAmountFirstRanking.Ranking == 1).show(truncate=False)

+----------+----------------------------+------------+-------+
|Month Name|Name                        |Total Amount|Ranking|
+----------+----------------------------+------------+-------+
|January   |The Barbecue Chicken Pizza  |3770.25     |1      |
|February  |The California Chicken Pizza|3537.00     |1      |
|March     |The Barbecue Chicken Pizza  |4049.25     |1      |
|April     |The Barbecue Chicken Pizza  |3788.50     |1      |
|May       |The Barbecue Chicken Pizza  |3953.50     |1      |
|June      |The California Chicken Pizza|3870.50     |1      |
|July      |The Thai Chicken Pizza      |4073.75     |1      |
|August    |The California Chicken Pizza|3977.25     |1      |
|September |The Thai Chicken Pizza      |3736.25     |1      |
|October   |The Thai Chicken Pizza      |3503.25     |1      |
|November  |The Barbecue Chicken Pizza  |3881.75     |1      |
|December  |The Thai Chicken Pizza      |3813.00     |1      |
+----------+----------------------------+------------+-

### Maximum Total Amount in month:

In [58]:
MaxTotalAmount = dfPizzaMonthNameTotalAmount.agg({"Total Amount": "max"}).collect()[0][0]

dfPizzaMonthNameTotalAmount.filter(dfPizzaMonthNameTotalAmount["Total Amount"] == MaxTotalAmount).show(truncate=False)

+----------+----------------------+------------+
|Month Name|Name                  |Total Amount|
+----------+----------------------+------------+
|July      |The Thai Chicken Pizza|4073.75     |
+----------+----------------------+------------+



In [59]:
#Looking for values with only one ingredient for example salami using wildcard
# Pizza without orders -> using case when
#Things to be considered in next project
# dealing with these https://www.datasciencemadesimple.com/get-month-year-and-quarter-from-date-in-pyspark/, 
# https://www.datasciencemadesimple.com/get-day-of-month-day-of-year-day-of-week-from-date-in-pyspark/
#consdering day of week? https://www.datasciencemadesimple.com/get-day-of-month-day-of-year-day-of-week-from-date-in-pyspark/
# browsing all materials that I have so as to select!!
## using UDF (put  example and remove digit at the beginning) 

# using pandas and look for spicy pizza or ingredients
#case when with size and is null where it is not available
#https://sparkbyexamples.com/pyspark/pyspark-groupby-explained-with-example/
# https://sparkbyexamples.com/pyspark/pyspark-window-functions/
# https://sparkbyexamples.com/pyspark/pyspark-udf-user-defined-function/ 
# Considering next transformations to be used using inspirations to run this project from .doc files 
## using pandas / pivot
# https://pub.towardsai.net/exploratory-data-analysis-eda-using-pyspark-b43e71fcec9f
#https://dev.to/kinyungu_denis/apache-pyspark-for-data-engineering-3phi